<a href="https://colab.research.google.com/github/lorenzopaoria/Smoking-detection-and-distance-analysis/blob/main/distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Find the distance between smoker and not

In [1]:
import cv2
import numpy as np
from dataclasses import dataclass
from typing import List, Tuple
import math

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
@dataclass
class Person:
    x1: int
    y1: int
    x2: int
    y2: int
    is_smoking: bool
    confidence: float

In [4]:
def extract_people_from_image(image_path: str) -> List[Person]:
    """
    Estrae le informazioni sulle persone dall'immagine usando le bounding box
    e la classificazione fumatore/non fumatore basata sui valori di confidenza
    """
    image = cv2.imread(image_path)
    height, width = image.shape[:2]

    # Questo è un esempio di come estrarre le informazioni.
    # Nella pratica, dovresti adattare questo codice per leggere i tuoi dati reali
    people = []

    # Per ogni bounding box rilevata
    # Esempio di come leggere i dati:
    # - Class 0 (verde) = fumatore
    # - Class 1/2 (blu/rosso) = non fumatore
    # Confidence values sono già presenti nelle box

    return people

In [5]:
def calculate_center_point(person: Person) -> Tuple[float, float]:
    """Calcola il punto centrale di una persona"""
    center_x = (person.x1 + person.x2) / 2
    center_y = (person.y1 + person.y2) / 2
    return (center_x, center_y)

def calculate_distance(p1: Person, p2: Person, pixels_per_meter: float = 100) -> float:
    """
    Calcola la distanza euclidea tra due persone
    pixels_per_meter è un fattore di conversione da calibrare in base alla tua setup
    """
    c1 = calculate_center_point(p1)
    c2 = calculate_center_point(p2)

    # Distanza euclidea in pixel
    pixel_distance = math.sqrt(
        (c2[0] - c1[0])**2 + (c2[1] - c1[1])**2
    )

    # Converti in metri
    return pixel_distance / pixels_per_meter

In [6]:
def analyze_distances(image_path: str) -> dict:
    """
    Analizza le distanze tra fumatori e non fumatori in un'immagine
    """
    people = extract_people_from_image(image_path)

    smokers = [p for p in people if p.is_smoking]
    non_smokers = [p for p in people if not p.is_smoking]

    distances = []
    for smoker in smokers:
        for non_smoker in non_smokers:
            distance = calculate_distance(smoker, non_smoker)
            distances.append({
                'smoker_idx': smokers.index(smoker),
                'non_smoker_idx': non_smokers.index(non_smoker),
                'distance': distance
            })

    return {
        'min_distance': min(d['distance'] for d in distances) if distances else float('inf'),
        'max_distance': max(d['distance'] for d in distances) if distances else 0,
        'avg_distance': sum(d['distance'] for d in distances) / len(distances) if distances else 0,
        'detailed_distances': distances
    }


In [7]:
def visualize_distances(image_path: str, analysis_result: dict):
    """
    Visualizza le distanze sull'immagine
    """
    image = cv2.imread(image_path)
    people = extract_people_from_image(image_path)

    # Disegna le bounding box e le linee di distanza
    for person in people:
        color = (0, 255, 0) if person.is_smoking else (0, 0, 255)
        cv2.rectangle(image, (person.x1, person.y1), (person.x2, person.y2), color, 2)

    # Disegna le linee di distanza
    for distance_info in analysis_result['detailed_distances']:
        smoker = people[distance_info['smoker_idx']]
        non_smoker = people[distance_info['non_smoker_idx']]

        s_center = calculate_center_point(smoker)
        ns_center = calculate_center_point(non_smoker)

        # Disegna una linea tra i centri
        cv2.line(image, (int(s_center[0]), int(s_center[1])), (int(ns_center[0]), int(ns_center[1])), (255, 255, 0), 1)

        # Aggiungi la distanza come testo
        mid_point = ((s_center[0] + ns_center[0])//2, (s_center[1] + ns_center[1])//2)
        cv2.putText(image,
                    f"{distance_info['distance']:.2f}m",
                    (int(mid_point[0]), int(mid_point[1])),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)

    cv2.imshow("Distances Analysis", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [8]:
def main():
    image_path = '/content/drive/MyDrive/test_trained_person'

    # Analizza le distanze
    analysis = analyze_distances(image_path)

    # Stampa i risultati
    print(f"Distanza minima: {analysis['min_distance']:.2f} metri")
    print(f"Distanza massima: {analysis['max_distance']:.2f} metri")
    print(f"Distanza media: {analysis['avg_distance']:.2f} metri")

    # Visualizza i risultati
    visualize_distances(image_path, analysis)

if __name__ == "__main__":
    main()

AttributeError: 'NoneType' object has no attribute 'shape'